In [ ]:
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import utils, models,datasets
import torchvision.transforms as transforms
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import time


/home/asineesh/I2I/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
train_transform = transforms.Compose([transforms.ColorJitter(brightness=.5, saturation=.25,hue=.1, contrast=.5),
                                      transforms.RandomAffine(10, (0.05, 0.05)),
                                      transforms.RandomHorizontalFlip(.5),
                                      transforms.RandomVerticalFlip(.5),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.6716241, 0.48636872, 0.60884315],
                                                           [0.27210504, 0.31001145, 0.2918652])
        ])

test_transform = transforms.Compose([        
            transforms.ToTensor(),
            transforms.Normalize([0.6716241, 0.48636872, 0.60884315],
                                 [0.27210504, 0.31001145, 0.2918652])
        ])

In [ ]:
train_data = datasets.ImageFolder(root="PCam/Pcam_Train/Pcam_Train",transform=train_transform)
len(train_data)

245760

In [ ]:
valid_data = datasets.ImageFolder(root="PCam/Pcam_Test_192/Pcam_Test_192",transform=test_transform)
len(valid_data)

40960

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=False)
model.fc = nn.Linear(2048,2).to(device)
model = model.to(device)

Using cache found in /home/asineesh/.cache/torch/hub/pytorch_vision_v0.6.0
/home/asineesh/I2I/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asineesh/I2I/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
torch.manual_seed(42)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True,num_workers=24)
test_loader = DataLoader(valid_data, batch_size=128, shuffle=True,num_workers=24)

In [ ]:
torch.backends.cudnn.benchmark = False

In [ ]:
start_time = time.time()     
    
train_losses = []
train_correct = []
test_correct = []
best_acc = 0.0
    
for epoch in range(20):
        trn_corr = 0
        tst_corr = 0
        loop = tqdm(train_loader)
                
        # Run the training batches
        for b, (X_train, y_train) in enumerate(loop):       
            
            X_train, y_train = X_train.to(device), y_train.to(device)
            y_pred = model(X_train)
            loss = criterion(y_pred, y_train)
     
            # Tally the number of correct predictions
            predicted = torch.max(y_pred.data, 1)[1]
            batch_corr = (predicted == y_train).sum()
            trn_corr += batch_corr
            
            # Update parameters
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loop.set_postfix(loss=loss.item())   
           
    
            train_losses.append(loss)
            train_correct.append(trn_corr)
        
        print(f'epoch: {epoch:2} Train accuracy: {train_correct[-1].item()*100/len(train_data):.3f}%')
        
        # Run the testing batches
        
        with torch.no_grad():
            for b, (X_test, y_test) in enumerate(test_loader):
                
                X_test, y_test = X_test.to(device), y_test.to(device)
                
                # Apply the model
                
                y_val = model(X_test)              
                
                # Tally the number of correct predictions
                predicted = torch.max(y_val.data, 1)[1] 
                tst_corr += (predicted == y_test).sum()
                test_correct.append(tst_corr)               
        
        
        print(f'epoch: {epoch:2} Test accuracy: {test_correct[-1].item()*100/len(valid_data):.3f}%')
        test_acc = test_correct[-1].item()*100/len(valid_data)
        if test_acc>best_acc:
            torch.save(model.state_dict(), "resnet50_"+str(test_acc)+"_"+str(epoch)+" .pt")
            best_acc = test_acc
       
        print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed    
    
print(f'Final Test accuracy: {test_correct[-1].item()*100/len(valid_data):.3f}%')

100%|██████████| 1920/1920 [11:07<00:00,  2.88it/s, loss=0.439]

epoch:  0 Train accuracy: 76.337%


epoch:  0 Test accuracy: 79.609%

Duration: 717 seconds


100%|██████████| 1920/1920 [16:58<00:00,  1.89it/s, loss=0.395]

epoch:  1 Train accuracy: 85.254%


...

epoch: 18 Train accuracy: 95.086%


epoch: 18 Test accuracy: 90.151%

Duration: 19114 seconds


100%|██████████| 1920/1920 [16:44<00:00,  1.91it/s, loss=0.222] 

epoch: 19 Train accuracy: 95.183%


epoch: 19 Test accuracy: 90.447%

Duration: 20172 seconds
Final Test accuracy: 90.447%
